In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [2]:
import pyspark.sql.functions as F

In [3]:
lending = spark.read.csv('accepted_2007_to_2018Q4.csv.gz', inferSchema=True, header=True)
lending.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

## Decide on categories for features
loan_amnt, funded_amnt, term, int_rate, grade, emp_length, home_ownership, annual_inc, addr_state?, fico_range_low (there's no point in using both FICO fields, they are close together and highly correlated), what else??

In [5]:
lending.select('loan_status').distinct().show() #shows different loan statuses

+--------------------+
|         loan_status|
+--------------------+
|          Fully Paid|
|             Default|
|                null|
|     In Grace Period|
|Does not meet the...|
|         Charged Off|
|            Oct-2015|
|  Late (31-120 days)|
|             Current|
|Does not meet the...|
|   Late (16-30 days)|
+--------------------+



Response variable is loan_status. We'll need to change to a binary with 'Default', 'Charged Off', and 'Does not meet the credit policy. Status:Charged Off' can be 1 for default and the others can be 0 for not defaulted. Null rows will need to be deleted.

In [4]:
df_train = lending.select('loan_amnt', 'term', 'int_rate', 'grade', 'emp_length',
                          'home_ownership', 'tot_cur_bal', 'total_pymnt', 'annual_inc', 'addr_state', 'fico_range_low')

In [5]:
df_train.take(3)

[Row(loan_amnt=3600.0, term=' 36 months', int_rate=13.99, grade='C', emp_length='10+ years', home_ownership='MORTGAGE', tot_cur_bal='144904.0', total_pymnt='4421.723916800001', annual_inc='55000.0', addr_state='PA', fico_range_low='675.0'),
 Row(loan_amnt=24700.0, term=' 36 months', int_rate=11.99, grade='C', emp_length='10+ years', home_ownership='MORTGAGE', tot_cur_bal='204396.0', total_pymnt='25679.66', annual_inc='65000.0', addr_state='SD', fico_range_low='715.0'),
 Row(loan_amnt=20000.0, term=' 60 months', int_rate=10.78, grade='B', emp_length='10+ years', home_ownership='MORTGAGE', tot_cur_bal='189699.0', total_pymnt='22705.924293878397', annual_inc='63000.0', addr_state='IL', fico_range_low='695.0')]

In [6]:
df_train.cache()

DataFrame[loan_amnt: double, term: string, int_rate: double, grade: string, emp_length: string, home_ownership: string, tot_cur_bal: string, total_pymnt: string, annual_inc: string, addr_state: string, fico_range_low: string]

In [7]:
#Find different term levels
df_train.select('term').distinct().show()

+----------+
|      term|
+----------+
| 36 months|
|      null|
| 60 months|
+----------+



In [12]:
# for each field, compute missing percentage

df_train.agg(*[
    (1 - F.count(c) / F.count('*')).alias(c + '_miss')
    for c in df_train.columns
]).show()

+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      loan_amnt_miss|           term_miss|       int_rate_miss|          grade_miss|    emp_length_miss| home_ownership_miss|    tot_cur_bal_miss|    total_pymnt_miss|     annual_inc_miss|     addr_state_miss| fico_range_low_miss|
+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|1.459724218288993...|1.459724218288993...|1.459724218288993...|1.459724218288993...|0.06499753837415911|1.459724218288993...|0.031054084551650152|2.034767092151668E-5|1.636660487169816...|1.503958285509199...|1.503958285509199...|
+--------------------+--------------------+--------------------+--------

In [21]:
df_train.printSchema()

root
 |-- loan_amnt: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_pymnt: float (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- fico_range_low: float (nullable = true)



In [11]:
# row count
print(f'rows={df_train.count()}')

rows=2260701


In [12]:
# distinct row count
print(f'rows={df_train.distinct().count()}')

rows=2259027


In [8]:
#Looks like there are duplicates so we will drop those
df_train = df_train.dropDuplicates()
print(f'rows={df_train.count()}')

rows=2260669


In [9]:
#Convert these columns since they are numbers but cast as strings in original data
df_train = df_train.withColumn("tot_cur_bal", df_train['tot_cur_bal'].cast('float'))
df_train = df_train.withColumn("total_pymnt", df_train["total_pymnt"].cast('float'))
df_train = df_train.withColumn("fico_range_low", df_train["fico_range_low"].cast('float'))

In [10]:
#Descriptive stats on numerical columns
df_train.describe('loan_amnt', 'int_rate', 'tot_cur_bal', 'total_pymnt').show()

+-------+------------------+------------------+------------------+------------------+
|summary|         loan_amnt|          int_rate|       tot_cur_bal|       total_pymnt|
+-------+------------------+------------------+------------------+------------------+
|  count|           2260668|           2260668|           2190484|           2260607|
|   mean|15046.931227849467|13.092829115111373|  142486.039012547|12081.553566208604|
| stddev| 9190.245488232784| 4.832138364571108|160691.95987801175| 9901.812291187154|
|    min|             500.0|              5.31|               0.0|               0.0|
|    max|           40000.0|             30.99|         9971659.0|          63296.88|
+-------+------------------+------------------+------------------+------------------+



In [14]:
df_train.groupBy('addr_state').count().orderBy('count', ascending=False).show(56)


+------------------+------+
|        addr_state| count|
+------------------+------+
|                CA|313835|
|                TX|186145|
|                NY|186137|
|                FL|161833|
|                IL| 91093|
|                NJ| 83063|
|                PA| 76893|
|                OH| 75085|
|                GA| 74159|
|                VA| 62931|
|                NC| 62708|
|                MI| 58740|
|                MD| 53985|
|                AZ| 53752|
|                MA| 51754|
|                CO| 48170|
|                WA| 47043|
|                MN| 39503|
|                IN| 37505|
|                MO| 36076|
|                CT| 35773|
|                TN| 35482|
|                NV| 32640|
|                WI| 29869|
|                SC| 27996|
|                AL| 27282|
|                OR| 26786|
|                LA| 25750|
|                KY| 21875|
|                OK| 20691|
|                KS| 19104|
|                AR| 17074|
|                UT|

It looks like there are some categories that aren't actually states, we'll need to get rid of those

In [25]:
#This uses a regex to find only state names on the 'addr_state' column
df_train = df_train.filter(df_train['addr_state'].rlike(r'^[A-Z]{2}?'))
#we can check that those few rows have been removed
df_train.count()


2260414

Looks like we lost about 200 rows but that's out of over two million

In [23]:
df_train.groupBy('addr_state').count().orderBy('count', ascending=False).show(66)

+--------------------+------+
|          addr_state| count|
+--------------------+------+
|                  CA|314481|
|                  NY|186374|
|                  TX|186318|
|                  FL|161977|
|                  IL| 91152|
|                  NJ| 83123|
|                  PA| 76930|
|                  OH| 75125|
|                  GA| 74186|
|                  VA| 62944|
|                  NC| 62730|
|                  MI| 58764|
|                  MD| 54000|
|                  AZ| 53769|
|                  MA| 51773|
|                  CO| 48178|
|                  WA| 47053|
|                  MN| 39513|
|                  IN| 37515|
|                  MO| 36082|
|                  CT| 35782|
|                  TN| 35483|
|                  NV| 32653|
|                  WI| 29873|
|                  SC| 28001|
|                  AL| 27282|
|                  OR| 26788|
|                  LA| 25757|
|                  KY| 21885|
|                  OK| 20691|
|         

In [36]:
#Find Outliers

def outliers(df):
    bnds = {}
    cols = ['loan_amnt', 'int_rate', 'tot_cur_bal', 'total_pmnt', 'fico_range_low']

    for col in cols:
        quantiles = df.approxQuantile(col, [0.25, 0.75], 0.05)
        IQR = quantiles[1] - quantiles[0]

        bnds[col] = [
         quantiles[0] - 1.5 * IQR,
         quantiles[1] + 1.5 * IQR
        ]
    return bnds